In [1]:
import pandas as pd
import random
import json
import itertools


pd.set_option('display.max_rows', 50)


from functions import sharpe_ratio_calculation, generate_rand_portfolios, select_top_five, run_min_variance

In [2]:
df_all_stocks = pd.read_csv('stocks_data.csv')
cryptos_df = pd.read_csv('cryptos_data.csv')

In [3]:
#Random Portfolios generation
tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)

In [4]:
#Select top five sharpe ratio portfolios from a portfolio
sharpe_ratio = sharpe_ratio_calculation(df_all_stocks, rf_rate_annual = 0.02)
top_five_dict = select_top_five(random_portfolios, metric=sharpe_ratio)

In [ ]:
for i in range(21,25):

    print('Doing', i)

    top_five_sets = dict(itertools.islice(top_five_dict.items(), i, i+1))
    results = run_min_variance(df_all_stocks, top_five_sets) 
    with open(f"my_dict{i}.json", "w") as f:
        json.dump(results, f, indent=4)

Doing 21
